In [1]:
from pyspark.sql import SparkSession

In [2]:
# Membuat SparkSession
spark = SparkSession.builder.master("local[1]")\
                    .appName('Pyspark job')\
                    .getOrCreate()

# Menampilkan versi PySpark
print('Pyspark Version : ' + spark.version)
print('Pyspark Version : ' + spark.sparkContext.version)

# Load Data SalesData
df = spark.read.csv('sales_data.csv',header=True,sep=";",inferSchema=True)
# temp table SalesData
df.createOrReplaceTempView("temp_salesdata")

# Load Data Zipssortedbycitystate
df2 = spark.read.csv('Zipssortedbycitystate.csv',header=True,sep=";",inferSchema=True)
# temp table Zipssortedbycitystate
df2.createOrReplaceTempView("temp_zipdata")


Pyspark Version : 3.4.0
Pyspark Version : 3.4.0


In [3]:
# cari postalcode yang null dari sales data
df = spark.sql("select * from temp_salesdata where postalcode is null")

# hapus postal code karena bernilai null atau kosong
df = df.drop("postalcode")

# buat temp baru untuk menyimpan hasil yang postalcode = null
df.createOrReplaceTempView("temp_postalcodenull")

# cari data yang postalcode nya tidak null
df2 = spark.sql("select * from temp_salesdata where postalcode is not null")

# simpan data yang postalcode nya tidak null ke temp baru
df2.createOrReplaceTempView("temp_postalnotcodenull")
df.show()
df2.show()

+-----------+---------------+---------+---------------+-------+----------------+-------+------+--------+-------+----------------+----+-----------+--------------------+----------+--------------------+------------+-------------+-----+-------------+---------+---------------+----------------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES|       ORDERDATE| STATUS|QTR_ID|MONTH_ID|YEAR_ID|     PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|     PHONE|        ADDRESSLINE1|ADDRESSLINE2|         CITY|STATE|      COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|
+-----------+---------------+---------+---------------+-------+----------------+-------+------+--------+-------+----------------+----+-----------+--------------------+----------+--------------------+------------+-------------+-----+-------------+---------+---------------+----------------+
|      10159|             49|    100.0|             14|5205.27|10/10/2003 00:00|Shipped|     4|      10|   2003|     Motorcycles| 

In [4]:
# sql dalam CTE_ZIPCODE merupakan proses mengubah digit terakhir postalcode sesuai selisih digit yang memiliki city dan state yang sama
# selanjutnya ialah memasukkan kolom POSTALCODE ke dalam temp_postalcodenull(table dengan postalcode == null) sesuai city dan state yang sama
df = spark.sql('''
            WITH CTE_ZIPCODE AS (
                    SELECT 
                        city, 
                        state, 
                        MAX(postalcode) - MIN(postalcode) AS jum,
                        LENGTH(CAST(MAX(postalcode) - MIN(postalcode) AS STRING)) AS jumlah_digit_diganti,
                        FIRST(postalcode) AS first_postalcode,
                        CONCAT(
                            SUBSTRING(FIRST(postalcode), 1, 5 - LENGTH(CAST(MAX(postalcode) - MIN(postalcode) AS STRING))),
                            REPEAT('X', LENGTH(CAST(MAX(postalcode) - MIN(postalcode) AS STRING)))
                        ) AS POSTALCODE
                    FROM temp_zipdata 
                    GROUP BY city, state
                )
                SELECT
                    tn.*,
                    cte.POSTALCODE
                FROM temp_postalcodenull tn
                JOIN CTE_ZIPCODE cte
                ON tn.city = cte.city AND tn.state = cte.state
                ''')
df.show()
# simpan hasil pengabungan ke dalam temp baru yaitu temp_hasil
df.createOrReplaceTempView("temp_hasil")

+-----------+---------------+---------+---------------+-------+----------------+-------+------+--------+-------+----------------+----+-----------+--------------------+----------+--------------------+------------+-------------+-----+-------+---------+---------------+----------------+----------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES|       ORDERDATE| STATUS|QTR_ID|MONTH_ID|YEAR_ID|     PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|     PHONE|        ADDRESSLINE1|ADDRESSLINE2|         CITY|STATE|COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|POSTALCODE|
+-----------+---------------+---------+---------------+-------+----------------+-------+------+--------+-------+----------------+----+-----------+--------------------+----------+--------------------+------------+-------------+-----+-------+---------+---------------+----------------+----------+
|      10209|             48|    44.69|              3|2145.12|01/09/2004 00:00|Shipped|     1|       1|   2004|   

In [8]:
# Mendapatkan daftar kolom dari temp_postalnotcodenull
columns_temp_postalnotcodenull = spark.table("temp_postalnotcodenull").columns

# Menentukan kolom secara eksplisit sesuai urutan dari temp_postalnotcodenull
df_union = spark.sql(f'''
    SELECT {', '.join(columns_temp_postalnotcodenull)}
    FROM temp_postalnotcodenull
    UNION
    SELECT {', '.join(columns_temp_postalnotcodenull)}
    FROM temp_hasil
''')
# outputHasil merupakan tabel akhir
df_union.createOrReplaceTempView("outputHasil")

# # csv
# df_union.write.csv('sales_csv/OutputSpark.csv', header=True, mode='overwrite')

df_union.show()

df2 = spark.sql("SELECT * FROM outputHasil where city = 'San Francisco'").show()
df_union.count()

+-----------+---------------+---------+---------------+-------+----------------+----------+------+--------+-------+------------+----+-----------+--------------------+---------------+--------------------+------------+-----------+-----+----------+-------+---------+---------------+----------------+
|ORDERNUMBER|QUANTITYORDERED|PRICEEACH|ORDERLINENUMBER|  SALES|       ORDERDATE|    STATUS|QTR_ID|MONTH_ID|YEAR_ID| PRODUCTLINE|MSRP|PRODUCTCODE|        CUSTOMERNAME|          PHONE|        ADDRESSLINE1|ADDRESSLINE2|       CITY|STATE|POSTALCODE|COUNTRY|TERRITORY|CONTACTLASTNAME|CONTACTFIRSTNAME|
+-----------+---------------+---------+---------------+-------+----------------+----------+------+--------+-------+------------+----+-----------+--------------------+---------------+--------------------+------------+-----------+-----+----------+-------+---------+---------------+----------------+
|      10153|             49|    100.0|             10|7036.89|  9/28/2003 0:00|   Shipped|     3|       9|  

2823